In [1]:
import IRQIV
import CSV, DataFrames

In [50]:
# Import GCP data
DH_survey = CSV.read(raw"/Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/CACO/2023/CornellDuckHarbor2023.csv", DataFrames.DataFrame)
GCPs_8_1_SC = DH_survey[40:64,:]
SC_8_1_bots = GCPs_8_1_SC[1:12,[1,7,8,9]] # Name, Lat, long, elev
SC_8_1_tops = GCPs_8_1_SC[14:25,[1,7,8,9]] # "

Row,Name,Longitude,Latitude,Ellipsoidal height
,String31,Float64,Float64,Float64
1,SC_stakes_top_01,-70.0762,41.9463,-24.683
2,SC_stakes_top_02,-70.0762,41.9463,-24.733
3,SC_stakes_top_03,-70.0762,41.9464,-24.785
4,SC_stakes_top_04,-70.0761,41.9464,-24.885
5,SC_stakes_top_06,-70.0761,41.9465,-24.901
6,SC_stakes_top_07,-70.0761,41.9465,-25.06
7,SC_stakes_top_08,-70.0761,41.9466,-25.025
8,SC_stakes_top_09,-70.076,41.9466,-24.993
9,SC_stakes_top_10,-70.076,41.9467,-24.986


In [35]:
size(GCPs_8_1_SC)

(25, 28)

In [31]:
GCPs_8_1_SC."Longitude"

25-element Vector{Float64}:
 -70.07622126
 -70.07619694
 -70.07615815
 -70.07612854
 -70.07609248
 -70.07607026
 -70.07606912
 -70.07604329
 -70.07601133
 -70.07603127
   ⋮
 -70.07612909
 -70.07609122
 -70.07607147
 -70.07606642
 -70.07604126
 -70.07600981
 -70.07602869
 -70.07608179
 -70.07622395

In [23]:
GCPs_8_1_SC[:,"Name"]

25-element Vector{InlineStrings.String31}:
 "SC_stakes_bot_01"
 "SC_stakes_bot_02"
 "SC_stakes_bot_03"
 "SC_stakes_bot_04"
 "SC_stakes_bot_06"
 "SC_stakes_bot_07"
 "SC_stakes_bot_08"
 "SC_stakes_bot_09"
 "SC_stakes_bot_10"
 "SC_stakes_bot_11"
 ⋮
 "SC_stakes_top_04"
 "SC_stakes_top_06"
 "SC_stakes_top_07"
 "SC_stakes_top_08"
 "SC_stakes_top_09"
 "SC_stakes_top_10"
 "SC_stakes_top_11"
 "SC_stakes_top_12"
 "SC_stakes_top_13"